In [121]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('labeled_videos.db')

# Загружаем данные о категориях (допустим у вас уже есть DataFrame с категориями и ключевыми словами)
conn = sqlite3.connect('../../data/professions.db')
categories_df = pd.read_sql('SELECT * FROM data', conn)

categories_df.head()

,name,category,desc,profile_codes,keywords,keywords_extention
0,Агроном,Агропромышленность,"Выращивает ягоды, овощи, фрукты, грибы, кормов...","35.02.05 - Агрономия, 35.03.03 - Агрохимия и а...","агрономия, выращивание, растения, удобрения, п...","агрономия, выращивание, растения, удобрения, п..."
1,Маркшейдер,"Архитектура, строительство и недвижимость","Исследует месторождения полезных ископаемых, п...","05.03.01 - Геология, 21.05.04 - Горное дело, 2...","маркшейдер, геодезия, исследование месторожден...","маркшейдер, геодезия, исследование месторожден..."
2,Реставратор,"Архитектура, строительство и недвижимость","Восстанавливает старинные здания, сооружения и...","07.02.01 - Архитектура, 54.02.04 - Реставрация...","реставрация, реставратор, консервация искусств...","реставрация, реставратор, консервация искусств..."
3,Слесарь по ремонту строительных машин,"Архитектура, строительство и недвижимость",Ремонтирует строительную технику<br><br>\n🔧 Ре...,23.02.03 - Техническое обслуживание и ремонт а...,"слесарь, ремонт строительных машин, техническо...","слесарь, ремонт строительных машин, техническо..."
4,Монтажник санитарно-технических систем и обору...,"Архитектура, строительство и недвижимость",Устанавливает и обслуживает системы водоснабже...,08.01.26 - Мастер по ремонту и обслуживанию ин...,"монтажник, сантехника, водоснабжение, канализа...","монтажник, сантехника, водоснабжение, канализа..."


ModuleNotFoundError: No module named 'google_auth_oauthlib'

In [124]:
videos_df['category_matches'] = ""

# Функция для подсчёта вхождений ключевых слов каждой категории
def match_categories(row, categories_df):
    category_matches = {}
    # Объединяем title, tags и description с добавлением пробелов в начале и в конце
    tags = row['tags'].replace('|', ' ').replace('\"', ' ')
    combined_text = f" {row['title']}  {tags}  {row['description']}  {row['channelTitle']} "
    for _, category_row in categories_df.iterrows():
        category_name = category_row['name']
        keywords = category_row['keywords_extention'].split(', ')
        for keyword in keywords:
            # Добавляем пробелы к каждому ключевому слову для точного поиска
            keyword_with_spaces = " " + keyword.lower() + " "
            matches = combined_text.count(keyword_with_spaces)
        if matches > 0:
            category_matches[category_name] = matches
    return category_matches if category_matches else {'Unknown': 0}

# Применяем функцию к каждой строке в DataFrame с видео
videos_df['category_matches'] = videos_df.apply(lambda row: match_categories(row, categories_df), axis=1)

In [125]:
# Создаем новые столбцы для категорий и количества вхождений
videos_df['categories'] = ""
videos_df['categories_cnt'] = ""

for idx, row in videos_df.iterrows():
    categories_str = ""
    cnt_str = ""
    category_matches = row['category_matches']
    for category_name, count in category_matches.items():
        categories_str += category_name + ","
        cnt_str += str(count) + ","
    # Обновляем DataFrame напрямую
    videos_df.at[idx, 'categories'] = categories_str.rstrip(',')
    videos_df.at[idx, 'categories_cnt'] = cnt_str.rstrip(',')

videos_df = videos_df.drop(columns=['category_matches'])

# Сохраняем результат
conn = sqlite3.connect('prof_with_categories.db')
videos_df.to_sql('data', conn, if_exists='replace', index=False)

conn.close()  # Не забываем закрыть соединение с базой данных

In [126]:
videos_df = videos_df[videos_df['categories'] != 'Unknown']
len(videos_df)

8843

In [127]:
# Инициализация словаря для подсчета
label_counts = {}

# Итерация по DataFrame
for idx, row in videos_df.iterrows():
    # Разделяем строки на списки категорий и их количеств
    categories_list = row['categories'].split(',')
    counts_list = row['categories_cnt'].split(',')
    
    # Убеждаемся, что списки имеют одинаковую длину
    if len(categories_list) == len(counts_list):
        # Итерация по категориям и их количествам
        for category, count in zip(categories_list, counts_list):
            if category:  # Проверяем, не пустая ли строка
                # Преобразуем count в число и добавляем к общему подсчету
                label_counts[category] = label_counts.get(category, 0) + int(count)

# Выводим результаты
for label, count in label_counts.items():
    print(f"{label}: {count}")


Бизнес-аналитик: 5054
Пресс-атташе: 2163
Радиоинженер: 4482
Грумер: 1826
Шеф-повар: 92
Звукооператор: 367
Фармацевт: 141
Швея-закройщица: 7
Регулировщик радиоэлектронной аппаратуры и приборов: 3
Врач: 18
Гейм-дизайнер: 13
Военнослужащий: 9
Риелтор: 25
